# SCADA Flow Rate Calculation Model

In [10]:
import pandas as pd
import os
import csv

import re
import datetime
import io
from zipfile import ZipFile

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Step 1: Load Scada Data

Instructions

1. Paste the name of the csv file into the df = pd.read_csv('') line. Make sure to put the file name between the quotes. Each lift station name will have to be pasted individually into this line. 

IMPORTANT: Run all the lines below this instruction box including the function before entering in the next lift station file in the df = pd.read_csv('') statement. The program needs to run in it's entirety for each individual file.

2. Once you have read in 1 individual file, run the following lines simply by clicking shift enter on each line. Stop when you reach the Lift Station Specifics heading.

## Read in SCADA Lifstation Data to a Dataframe

In [11]:
df = pd.read_csv('Kelly Island Data Edited.csv')

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_9174/3875305116.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Kelly Island Data Edited.csv')


## Convert the DateAndTime column to Datetime Format

In [12]:
df['DateAndTime'] = pd.to_datetime(df['DateAndTime'], format="%m/%d/%Y %H:%M:%S.%f", errors = 'coerce')


## Format the dataframe

In [13]:
df.drop('Status', axis=1, inplace=True)
df.drop('Marker', axis=1, inplace=True)
df.drop('Unnamed: 6', axis=1, inplace=True)
df.drop('Unnamed: 7', axis=1, inplace=True)

In [14]:
df = df.dropna()

In [15]:
#df.head(20)

## Remove Extraneous 1's and 0's

In [16]:
# first remove 0's with the same timestamp right next to each other

for i in reversed(range(1, len(df))):
    if df.loc[i, 'Val'] == 0 and df.loc[i-1, 'Val'] == 0 and df.loc[i, 'DateAndTime'] == df.loc[i-1, 'DateAndTime']:
        # drop both rows if conditions are met
        df = df.drop([i, i-1]).reset_index(drop=True)
    

## Set up our Calculated Dataframe 

In [17]:
calc_df = pd.DataFrame()
calc_df['LiftStationID'] = ()
calc_df['StartDateAndTime'] = ()
calc_df['EndDateAndTime'] = ()
calc_df['StartUnix'] = float()
calc_df['EndUnix'] = float()
calc_df['Time_diff'] = ()
calc_df['Pump_num'] = ()
calc_df['Elevation_dif'] = ()
calc_df['volume_per_foot'] = ()
calc_df['observed_pumping_flow_rate'] = int()
calc_df['avg_flow_rate_pump'] = int()
calc_df['average_observed_pumping_flow_rate'] = int()
calc_df['Time_to_Fill'] = float()
calc_df['Inflow'] = float()
calc_df['Avg_Inflow'] = float()
calc_df['Actual_Pump_Flow_Rate'] = float()
calc_df['Average_Pump_1_Flow_Rate'] = int()
calc_df['Average_Pump_2_Flow_Rate'] = int()
calc_df['Average_Pump_Flow_Rate(1&2)'] = int()

# Step 2: Lift Station Specifics

### Below are the specific figures for each lift station's wet well diameter and elevation difference. You will need to choose the corresponding diameter and elevation difference for the specific lift station data you read into the program in step 1.

| **Lift Station** | **liftID** | **wet well diameter** | **elevation difference** | **pump num.** |
|:----------------:|:----------:|:--------------------:|:------------------------:|:-------------:|
|   Bueno Vista    |     BV     |           5          |             -            |       2       |
|  Canyon River    |     CR     |           5          |             -            |       2       |
|   Caras Park     |     CP     |           10         |             -            |       2       |
|Community Hospital|     CH     |           8          |             -            |       2       |
| Dickens Street   |     DS     |           5          |             -            |       2       |
|  East Broadway   |     EB     |           8          |             -            |       2       |
|  East Missoula   |     EM     |           8          |             -            |       2       |
|     Futurity     |     FT     |           8          |             -            |       2       |
|   Kelly Island   |     KI     |           8          |             -            |       2       |
|Linda Vista GC    |     LVG    |          8 (check)   |             -            |       2       |
| Linda Vista LS   |     LGL    |          8 (check)   |             -            |       2       |
|      Mastad      |     MD     |           8          |             -            |       2       |
|Lower Miller Creek|     LMC    |           8          |             -            |       4       |
|    Momont #1     |     MT1    |           6          |             -            |       2       |
|    Momont #2     |     MT2    |           6          |             -            |       2       |
| Reserve Street   |     RS     |          12          |             -            |       4       |
|  Stratford 2662  |     SF     |            -         |             -            |       -       |
|   Third Street   |     TS     |           8          |             -            |       2       |
|    University    |     UT     |           7          |             -            |       2       |
|       Waldo      |     WD     |           8          |             -            |       2       |



# Step 3: Choosing Lift Station Specifics

At this point the SCADA file is formatted and loaded into our program. Now let's run the Flow Rate Function to create our new dataframe with the flow rates calculated.

1. Locate the specific lift station in the Lift Station Specifics table above. 

2. You will need Lifstation ID, Wet Well Diameter, and Elevation Difference values for your specific lift station.

# Flow Rate Function

In [18]:
def flow_rate_function (df, liftstation_id, well_diameter, elev_diff):
    
    calc_df = pd.DataFrame()
    calc_df['LiftStationID'] = ()
    calc_df['StartDateAndTime'] = ()
    calc_df['EndDateAndTime'] = ()
    calc_df['StartUnix'] = float()
    calc_df['EndUnix'] = float()
    calc_df['Time_diff'] = ()
    calc_df['Pump_num'] = ()
    calc_df['Elevation_dif'] = ()
    calc_df['volume_per_foot'] = ()
    calc_df['observed_pumping_flow_rate'] = int()
    calc_df['avg_flow_rate_pump'] = int()
    calc_df['average_observed_pumping_flow_rate'] = int()
    calc_df['Time_to_Fill'] = float()
    calc_df['Inflow'] = float()
    calc_df['Avg_Inflow'] = float()
    calc_df['Actual_Pump_Flow_Rate'] = float()
    calc_df['Average_Pump_1_Flow_Rate'] = int()
    calc_df['Average_Pump_2_Flow_Rate'] = int()
    calc_df['Average_Pump_Flow_Rate(1&2)'] = int()
    
    
    #Here let's fill calc_df with On and Off times + Pump Numbers
    
    has_date = False
    last_index = -1
    on = 1
    off = 0

    for index, row in df.iterrows():
        if row['Val'] == 1:
            tag = row['TagIndex']
            calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
            last_index += 1
            calc_df.at[last_index, 'Pump_num'] = tag
            has_date = True

        if row['Val'] == 0 and has_date:
            tag = row['TagIndex']
            calc_df.at[last_index, 'EndDateAndTime'] = row['DateAndTime']
            calc_df.at[last_index, 'Pump_num'] = tag
    
    calc_df['EndDateAndTime'] = pd.to_datetime(calc_df['EndDateAndTime'])
    
    #Let's delete any rows that don't have either a start or end time
    calc_df = calc_df.dropna(subset=['StartDateAndTime', 'EndDateAndTime'])
    # reset the index
    calc_df = calc_df.reset_index(drop=True)
    
    #delete any rows that have erronious pump numbers
    calc_df = calc_df[(calc_df['Pump_num'] == 974) | (calc_df['Pump_num'] == 975)]
    calc_df = calc_df.reset_index(drop=True)
    
    #Now let's get Unix time for each timestamp
    calc_df['StartUnix'] = pd.to_datetime(calc_df['StartDateAndTime']).astype(int)/ 10**9
    calc_df['EndUnix'] = pd.to_datetime(calc_df['EndDateAndTime']).astype(int) / 10**9

    #Calculating the time difference between Pumps turning on and off
    for index,row in calc_df.iterrows():
        result = (row['EndUnix']- row['StartUnix'])/60
        calc_df.at[index, 'Time_diff'] = result

    #Add in elevation difference
    calc_df['Elevation_dif'] = elev_diff
    #Calculate volume per foot

    vol_per_foot = ((well_diameter*well_diameter) * 3.14)/4*7.48
    vol_per_foot = round(vol_per_foot)
    calc_df['volume_per_foot'] = vol_per_foot

    #Now let's calculate our observed pumping Flow Rate
    for index, row in calc_df.iterrows():
        result = round((row['Elevation_dif'] * row['volume_per_foot'])/row['Time_diff'])
        calc_df.at[index,'observed_pumping_flow_rate'] = result

    #Now Calculate Average Flow Rate Per Pump
    cum_sum_1 =0
    cum_sum_2 =0
    count_1 =0
    count_2 =0

    for index, row in calc_df.iterrows():
        if row['Pump_num'] == 975:
            cum_sum_1 += row['observed_pumping_flow_rate']
            count_1 += 1
            calc_df.at[index, 'avg_flow_rate_pump'] = cum_sum_1 / count_1
        if row['Pump_num'] == 974:
            cum_sum_2 += row['observed_pumping_flow_rate']
            count_2 += 1
            calc_df.at[index, 'avg_flow_rate_pump'] = cum_sum_2 / count_2

    #Calculate average observed pumping flow rate (GPM)
    cum_sum = 0
    count = 0

    for index, row in calc_df.iterrows():
        cum_sum += row['avg_flow_rate_pump']
        count += 1
        calc_df.at[index, 'average_observed_pumping_flow_rate'] = cum_sum / count

    #Calculate time to fill
    #Get a list of timestamps when the pump turned off
    stop_list= []

    for index, row in calc_df.iterrows():
        stop = row['EndUnix']
        stop_list.append(stop)

    #Get a list of timestamps when the pump turned on    
    start_list = []

    for index, row in calc_df.iloc[1:].iterrows():
        start = row['StartUnix']
        start_list.append(start)
        
    #calculate the difference between the start and end time
    index = 1

    for start, end in zip(start_list, stop_list):
        result = (start - end) /60
        #print(result)
        calc_df.at[index, 'Time_to_Fill'] = result
        index += 1
        
    #Change any NaN's to 0    
    calc_df = calc_df.fillna(0)

        
    #Calculate Inflow
    for index, row in calc_df.iterrows():
        if row['Time_to_Fill'] != 0:
            inflow = row['volume_per_foot'] * row['Elevation_dif'] / row['Time_to_Fill']
            calc_df.at[index, 'Inflow'] = inflow
        else:
            pass
        
    #Calculate Average Inflow
    cum_sum = 0
    count = 0

    for index, row in calc_df.iterrows():
        cum_sum += row['Inflow']
        count += 1
        calc_df.at[index, 'Avg_Inflow'] = cum_sum / count
        
    #Calculate Actual Pump Flow Rate (GPM)
    for index, row in calc_df.iterrows():
        result = round(row['observed_pumping_flow_rate'] + row['Inflow'])
        calc_df.at[index, 'Actual_Pump_Flow_Rate'] = result
        
    #calculate our Average Flow Rate Per Pump
    #Pump 1
    cum_sum = 0
    count = 0

    for index, row in calc_df.iterrows():
        if row['Pump_num'] == 974:
            cum_sum += row['Actual_Pump_Flow_Rate']
            count += 1
            calc_df.at[index,'Average_Pump_1_Flow_Rate'] = cum_sum / count

    #Pump 2
    cum_sum = 0
    count = 0

    for index, row in calc_df.iterrows():
        if row['Pump_num'] == 975:
            cum_sum += row['Actual_Pump_Flow_Rate']
            count += 1
            calc_df.at[index,'Average_Pump_2_Flow_Rate'] = cum_sum / count
            
    #Calculate Average Pump Flow Rate in GPM
    cum_sum = 0
    count = 0

    for index, row in calc_df.iterrows():
        cum_sum += row['Actual_Pump_Flow_Rate']
        count += 1
        calc_df.at[index,'Average_Pump_Flow_Rate(1&2)'] = cum_sum / count
   
    calc_df['LiftStationID'] = liftstation_id

    return(calc_df)



 
 

# Step 4: Running the Function

You are now ready to run the function using the loaded SCADA data, and the lift station specifics from the table above.

1. In the Plug In line below, enter the appropriate values for your specific liftstation, into the function.

ex: Dickens Street 

calc_df = calc_df.append(flow_rate_function(df, liftstation_id="DS", well_diameter=5, elev_diff=2.2))

IMPORTANT: You only need to change the values for liftstation_id, well_diameter, and elev_diff.

2. Once you have updated the values in the function, click shift enter to run. The function will run and append the results to calc_df.

### Plug In

In [31]:
calc_df = calc_df.append(flow_rate_function(df, liftstation_id="BV", well_diameter=5, elev_diff=2.5))

In [32]:
#calc_df

# Step 5: Re Run or Export 

This is an either or step, at this point the function has run for your specific lift station, and the flow rate data is calculated and ready for export for said lift station.

IMPORTANT: 

If you are ready to export your data please follow instructions in section 1.A.

If you would like to calculate flow rate data for another lift station and add that data onto the current export please follow instructions in section 1.B.

1.A 
1. To export your calculated flow rate data, run the line below titled export.

1.B
1. If you want to add another liftstation to your output file, you will need to head back up to step 1.
2. Change the file name in the , "Read in SCADA Lifstation Data to a Dataframe", line. Remember you only need to change the file name in quotes.
3. Continue through steps 1-5 again, repeat as necessary.

### Export

In [30]:
calc_df.to_csv("Model Data for City of Missoula Liftstations.csv")